<a href="https://colab.research.google.com/github/kumar-abhishek/handson-ml2/blob/master/Final_all_fixes_BachChorales_HandsOnChapter_15.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Q10[part-1]. Download the Bach chorales dataset and unzip it. It is composed of 382 chorales composed by Johann Sebastian Bach. Each chorale is 100 to 640 time steps long, and each time step contains 4 integers, where each integer corresponds to a note’s index on a piano (except for the value 0, which means that no note is played). Train a model—recurrent, convolutional, or both—that can predict the next time step (four notes), given a sequence of time steps from a chorale. **bold text**

In [0]:
from google.colab import files
uploaded = files.upload()

Saving chorale_229.csv to chorale_229.csv


In [0]:
uploaded_validation = files.upload()

Saving chorale_000.csv to chorale_000.csv


In [36]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow import keras
import numpy as np

X_train_000 = pd.read_csv('chorale_000.csv')
print(X_train_000.head(10))

X_valid_229 = pd.read_csv('chorale_229.csv')
print(X_valid_229.head(10))

X_train_000 = X_train_000.to_numpy()
X_valid_229 = X_valid_229.to_numpy()

dataX=X_train_000[:-1]
dataY=X_train_000[1:]
dataValidationX=X_valid_229[:-1]
dataValidationY=X_valid_229[1:]

print(dataX[0:10])
print(dataY[0:10])
print(type(dataX))

scaler = MinMaxScaler( feature_range=(0, 1) )
dataXScaler = scaler.fit_transform(dataX)
dataYScaler = scaler.fit_transform(dataY)
print(dataXScaler[0:10])
print(dataYScaler[0:10])

inversedX = scaler.inverse_transform(dataXScaler[0:10])
print('inversed:')
print(inversedX[0:10])
print(scaler.inverse_transform(dataXScaler)[2:30])

scalerValidation = MinMaxScaler( feature_range=(0, 1) )
dataValidationX = scalerValidation.fit_transform(dataValidationX)
dataValidationY = scalerValidation.fit_transform(dataValidationY)
print(dataValidationX[0:5])
print(dataValidationY[0:5])


print(scaler.inverse_transform(dataXScaler)[2:30])


   note0  note1  note2  note3
0     74     70     65     58
1     74     70     65     58
2     74     70     65     58
3     74     70     65     58
4     75     70     58     55
5     75     70     58     55
6     75     70     60     55
7     75     70     60     55
8     77     69     62     50
9     77     69     62     50
   note0  note1  note2  note3
0     72     67     60     48
1     72     67     60     48
2     72     67     60     48
3     72     67     60     48
4     72     67     64     48
5     72     67     64     48
6     72     67     64     50
7     72     67     64     50
8     72     67     64     52
9     72     67     64     52
[[74 70 65 58]
 [74 70 65 58]
 [74 70 65 58]
 [74 70 65 58]
 [75 70 58 55]
 [75 70 58 55]
 [75 70 60 55]
 [75 70 60 55]
 [77 69 62 50]
 [77 69 62 50]]
[[74 70 65 58]
 [74 70 65 58]
 [74 70 65 58]
 [75 70 58 55]
 [75 70 58 55]
 [75 70 60 55]
 [75 70 60 55]
 [77 69 62 50]
 [77 69 62 50]
 [77 69 62 50]]
<class 'numpy.ndarray'>
[[0.44444444 0

In [55]:
# convert into input/output
i,j=0,0
k,l=0,0
X, y=[], []
validX, validY=[], []

while i<len(dataXScaler):
  X.append(np.array([dataXScaler[i]]))
  i += 1

while j<len(dataYScaler):
  y.append(dataYScaler[j])
  j+= 1

while k<len(dataValidationX):
  validX.append(np.array([dataValidationX[k]]))
  k += 1

while l<len(dataValidationY):
  validY.append(dataValidationY[l])
  l+=1

X=np.asarray(X)
y=np.asarray(y)
print(X[0:2])
print(y[0])
print(X.shape, y.shape)


validX=np.asarray(validX)
validY=np.asarray(validY)
print(validX[0:2])
print(validY[0])
print(validX.shape, validY.shape)

[[[0.44444444 0.77777778 1.         0.88235294]]

 [[0.44444444 0.77777778 1.         0.88235294]]]
[0.44444444 0.77777778 1.         0.88235294]
(191, 1, 4) (191, 4)
[[[0.41666667 0.375      0.41666667 0.2       ]]

 [[0.41666667 0.375      0.41666667 0.2       ]]]
[0.41666667 0.375      0.41666667 0.2       ]
(195, 1, 4) (195, 4)


In [59]:
from keras.models import Sequential
from keras import optimizers
from keras.layers import LSTM
from keras.layers import Dense, Dropout, BatchNormalization, TimeDistributed
# the dataset knows the number of features, e.g. 2
n_features = 4

# choose a number of time steps
n_steps = None

model = Sequential()
model.add(TimeDistributed(Dense(128), input_shape=(None, n_features)) ) # This line makes a lot of difference but why?
model.add(LSTM(64, input_shape=(None, n_features), return_sequences=True))
#model.add(BatchNormalization())
model.add(Dropout(0.2))

model.add(LSTM(32))
#model.add(BatchNormalization())
model.add(Dropout(0.2))
model.add(Dense(n_features))
model.compile(optimizer='adam', loss='mse')

# fit model
model.fit(X, y, epochs=200, validation_data=(validX, validY))

Train on 191 samples, validate on 195 samples
Epoch 1/200
191/191 [==============================] - 12s 61ms/step - loss: 0.2875 - val_loss: 0.2520
Epoch 2/200
191/191 [==============================] - 0s 363us/step - loss: 0.2435 - val_loss: 0.2039
Epoch 3/200
191/191 [==============================] - 0s 332us/step - loss: 0.1859 - val_loss: 0.1397
Epoch 4/200
191/191 [==============================] - 0s 366us/step - loss: 0.1156 - val_loss: 0.0740
Epoch 5/200
191/191 [==============================] - 0s 389us/step - loss: 0.0622 - val_loss: 0.0537
Epoch 6/200
191/191 [==============================] - 0s 394us/step - loss: 0.0588 - val_loss: 0.0574
Epoch 7/200
191/191 [==============================] - 0s 369us/step - loss: 0.0594 - val_loss: 0.0485
Epoch 8/200
191/191 [==============================] - 0s 370us/step - loss: 0.0515 - val_loss: 0.0467
Epoch 9/200
191/191 [==============================] - 0s 381us/step - loss: 0.0491 - val_loss: 0.0460
Epoch 10/200
191/191 [=====

In [60]:
# demonstrate prediction
print(scaler.inverse_transform(dataXScaler)[2:30])

x_input = dataXScaler[2:30]
x_input = x_input.reshape((1, len(x_input), 4))
print(scaler.inverse_transform(dataXScaler)[2:30])
yhat = model.predict(x_input, verbose=0)
print(scaler.inverse_transform(yhat))
print('expected: ', scaler.inverse_transform(dataYScaler)[30])

[[74. 70. 65. 58.]
 [74. 70. 65. 58.]
 [75. 70. 58. 55.]
 [75. 70. 58. 55.]
 [75. 70. 60. 55.]
 [75. 70. 60. 55.]
 [77. 69. 62. 50.]
 [77. 69. 62. 50.]
 [77. 69. 62. 50.]
 [77. 69. 62. 50.]
 [77. 70. 62. 55.]
 [77. 70. 62. 55.]
 [77. 69. 62. 55.]
 [77. 69. 62. 55.]
 [75. 67. 63. 48.]
 [75. 67. 63. 48.]
 [75. 69. 63. 48.]
 [75. 69. 63. 48.]
 [74. 70. 65. 46.]
 [74. 70. 65. 46.]
 [74. 70. 65. 46.]
 [74. 70. 65. 46.]
 [72. 69. 65. 53.]
 [72. 69. 65. 53.]
 [72. 69. 65. 53.]
 [72. 69. 65. 53.]
 [72. 69. 65. 53.]
 [72. 69. 65. 53.]]
[[74. 70. 65. 58.]
 [74. 70. 65. 58.]
 [75. 70. 58. 55.]
 [75. 70. 58. 55.]
 [75. 70. 60. 55.]
 [75. 70. 60. 55.]
 [77. 69. 62. 50.]
 [77. 69. 62. 50.]
 [77. 69. 62. 50.]
 [77. 69. 62. 50.]
 [77. 70. 62. 55.]
 [77. 70. 62. 55.]
 [77. 69. 62. 55.]
 [77. 69. 62. 55.]
 [75. 67. 63. 48.]
 [75. 67. 63. 48.]
 [75. 69. 63. 48.]
 [75. 69. 63. 48.]
 [74. 70. 65. 46.]
 [74. 70. 65. 46.]
 [74. 70. 65. 46.]
 [74. 70. 65. 46.]
 [72. 69. 65. 53.]
 [72. 69. 65. 53.]
 [72. 69. 6

**Q10[part-2] Then use this model to generate Bach-like music, one note at a time: you can do this by giving the model the start of a chorale and asking it to predict the next time step, then appending these time steps to the input sequence and asking the model for the next note, and so on. Also make sure to check out Google’s Coconet model, which was used for a nice Google doodle about Bach.**

In [61]:
# choose a number of time steps
n_steps = None
# convert into input/output
i=1
n=1
x_input = np.array([dataXScaler[0]])

x_input = x_input.reshape((1, len(x_input), n_features))
while i<len(dataXScaler):
  # demonstrate prediction
  print('Input: ')
  for input in x_input:
    print(scaler.inverse_transform(input))
  print('---------------')
  yhat = model.predict(x_input, verbose=1)
  print('Predicted Output: ', scaler.inverse_transform(yhat))
  print('expected: ', scaler.inverse_transform(dataYScaler)[i])
  print('\n\n')

  yhat = yhat.reshape((1, len(yhat), n_features))
  x_input = np.concatenate([x_input, yhat], axis=1)
  i += 1
  if i>20:
    break

Input: 
[[74. 70. 65. 58.]]
---------------
1/1 [==============================] - 0s 6ms/step
Predicted Output:  [[74.54715  69.76618  63.121742 56.327858]]
expected:  [74. 70. 65. 58.]



Input: 
[[74.         70.         65.         58.        ]
 [74.54714358 69.76617962 63.12173986 56.32785881]]
---------------
1/1 [==============================] - 0s 3ms/step
Predicted Output:  [[77.23478  73.9541   68.32366  62.923805]]
expected:  [74. 70. 65. 58.]



Input: 
[[74.         70.         65.         58.        ]
 [74.54714358 69.76617962 63.12173986 56.32785881]
 [77.23478138 73.95410085 68.3236618  62.92380428]]
---------------
1/1 [==============================] - 0s 4ms/step
Predicted Output:  [[79.70574  76.36403  69.889984 65.56413 ]]
expected:  [75. 70. 58. 55.]



Input: 
[[74.         70.         65.         58.        ]
 [74.54714358 69.76617962 63.12173986 56.32785881]
 [77.23478138 73.95410085 68.3236618  62.92380428]
 [79.70574284 76.36403131 69.88998318 65.56412959]]


In [67]:
from keras.layers import Dense, SimpleRNN

# Using SimpleRNN 
print(n_features)
model_rnn = Sequential()
model_rnn.add(TimeDistributed(Dense(128), input_shape=(None, n_features)))
model_rnn.add(SimpleRNN(100, input_shape=[None, n_features], return_sequences=True ))
model_rnn.add(SimpleRNN(100))
model_rnn.add(Dense(n_features))

model_rnn.compile(optimizer='adam', loss='mse')
model_rnn.fit(X, y, epochs=200, validation_data=(validX, validY))

4
Train on 191 samples, validate on 195 samples
Epoch 1/200
191/191 [==============================] - 12s 63ms/step - loss: 0.1649 - val_loss: 0.0683
Epoch 2/200
191/191 [==============================] - 0s 223us/step - loss: 0.0653 - val_loss: 0.0404
Epoch 3/200
191/191 [==============================] - 0s 208us/step - loss: 0.0267 - val_loss: 0.0297
Epoch 4/200
191/191 [==============================] - 0s 240us/step - loss: 0.0278 - val_loss: 0.0274
Epoch 5/200
191/191 [==============================] - 0s 205us/step - loss: 0.0179 - val_loss: 0.0185
Epoch 6/200
191/191 [==============================] - 0s 206us/step - loss: 0.0167 - val_loss: 0.0196
Epoch 7/200
191/191 [==============================] - 0s 227us/step - loss: 0.0153 - val_loss: 0.0170
Epoch 8/200
191/191 [==============================] - 0s 215us/step - loss: 0.0151 - val_loss: 0.0183
Epoch 9/200
191/191 [==============================] - 0s 210us/step - loss: 0.0148 - val_loss: 0.0167
Epoch 10/200
191/191 [===

In [69]:
# demonstrate prediction
x_input = dataXScaler[2:30]
x_input = x_input.reshape((1, len(x_input), 4))
print(scaler.inverse_transform(dataXScaler)[2:30])
yhat = model_rnn.predict(x_input, verbose=0)
print(scaler.inverse_transform(yhat))
print('expected: ', scaler.inverse_transform(dataYScaler)[30])

[[74. 70. 65. 58.]
 [74. 70. 65. 58.]
 [75. 70. 58. 55.]
 [75. 70. 58. 55.]
 [75. 70. 60. 55.]
 [75. 70. 60. 55.]
 [77. 69. 62. 50.]
 [77. 69. 62. 50.]
 [77. 69. 62. 50.]
 [77. 69. 62. 50.]
 [77. 70. 62. 55.]
 [77. 70. 62. 55.]
 [77. 69. 62. 55.]
 [77. 69. 62. 55.]
 [75. 67. 63. 48.]
 [75. 67. 63. 48.]
 [75. 69. 63. 48.]
 [75. 69. 63. 48.]
 [74. 70. 65. 46.]
 [74. 70. 65. 46.]
 [74. 70. 65. 46.]
 [74. 70. 65. 46.]
 [72. 69. 65. 53.]
 [72. 69. 65. 53.]
 [72. 69. 65. 53.]
 [72. 69. 65. 53.]
 [72. 69. 65. 53.]
 [72. 69. 65. 53.]]
[[75.66913 67.57323 61.02341 40.88458]]
expected:  [72. 69. 65. 53.]


In [70]:
# choose a number of time steps
n_steps = None
# convert into input/output
i=1
n=1
x_input = np.array([dataXScaler[0]])

x_input = x_input.reshape((1, len(x_input), n_features))
while i<len(dataXScaler):
  # demonstrate prediction
  print('Input: ')
  for input in x_input:
    print(scaler.inverse_transform(input))
  print('---------------')
  yhat = model.predict(x_input, verbose=1)
  print('Predicted Output: ', scaler.inverse_transform(yhat))
  print('expected: ', scaler.inverse_transform(dataYScaler)[i])
  print('\n\n')

  yhat = yhat.reshape((1, len(yhat), n_features))
  x_input = np.concatenate([x_input, yhat], axis=1)
  i += 1
  if i>20:
    break

Input: 
[[74. 70. 65. 58.]]
---------------
1/1 [==============================] - 0s 4ms/step
Predicted Output:  [[74.54715  69.76618  63.121742 56.327858]]
expected:  [74. 70. 65. 58.]



Input: 
[[74.         70.         65.         58.        ]
 [74.54714358 69.76617962 63.12173986 56.32785881]]
---------------
1/1 [==============================] - 0s 3ms/step
Predicted Output:  [[77.23478  73.9541   68.32366  62.923805]]
expected:  [74. 70. 65. 58.]



Input: 
[[74.         70.         65.         58.        ]
 [74.54714358 69.76617962 63.12173986 56.32785881]
 [77.23478138 73.95410085 68.3236618  62.92380428]]
---------------
1/1 [==============================] - 0s 3ms/step
Predicted Output:  [[79.70574  76.36403  69.889984 65.56413 ]]
expected:  [75. 70. 58. 55.]



Input: 
[[74.         70.         65.         58.        ]
 [74.54714358 69.76617962 63.12173986 56.32785881]
 [77.23478138 73.95410085 68.3236618  62.92380428]
 [79.70574284 76.36403131 69.88998318 65.56412959]]


In [66]:
# using GRU
from keras.models import Sequential
from keras.layers import GRU, Dense

# choose a number of time steps
n_steps = None
n_features = 4



model_gru = Sequential()
model_gru.add(TimeDistributed(Dense(128), input_shape=(None, n_features)))
model_gru.add(GRU(64, activation='tanh', input_shape=(None, n_features), return_sequences=True))
model_gru.add(BatchNormalization())

model_gru.add(GRU(32 , activation = 'tanh'))
model_gru.add(BatchNormalization())
model_gru.add(Dropout(0.2))
model_gru.add(Dense(n_features))
model_gru.compile(optimizer='adam', loss='mse')

# fit model
#model_gru.fit_generator(generator, epochs=500, validation_data=validation_generator)
model_gru.fit(X, y, epochs=200, validation_data=(validX, validY))

Train on 191 samples, validate on 195 samples
Epoch 1/200
191/191 [==============================] - 13s 69ms/step - loss: 1.5210 - val_loss: 0.9882
Epoch 2/200
191/191 [==============================] - 0s 386us/step - loss: 0.8841 - val_loss: 1.2071
Epoch 3/200
191/191 [==============================] - 0s 352us/step - loss: 0.7568 - val_loss: 0.8654
Epoch 4/200
191/191 [==============================] - 0s 375us/step - loss: 0.6692 - val_loss: 0.4700
Epoch 5/200
191/191 [==============================] - 0s 366us/step - loss: 0.5930 - val_loss: 0.4772
Epoch 6/200
191/191 [==============================] - 0s 366us/step - loss: 0.5898 - val_loss: 0.4964
Epoch 7/200
191/191 [==============================] - 0s 400us/step - loss: 0.5719 - val_loss: 0.3791
Epoch 8/200
191/191 [==============================] - 0s 350us/step - loss: 0.4966 - val_loss: 0.2142
Epoch 9/200
191/191 [==============================] - 0s 377us/step - loss: 0.5391 - val_loss: 0.1964
Epoch 10/200
191/191 [=====

In [71]:
# demonstrate prediction
x_input = dataXScaler[2:30]
x_input = x_input.reshape((1, len(x_input), 4))
print(scaler.inverse_transform(dataXScaler)[2:30])
yhat = model_gru.predict(x_input, verbose=0)
print(scaler.inverse_transform(yhat))
print('expected: ', scaler.inverse_transform(dataYScaler)[30])

[[74. 70. 65. 58.]
 [74. 70. 65. 58.]
 [75. 70. 58. 55.]
 [75. 70. 58. 55.]
 [75. 70. 60. 55.]
 [75. 70. 60. 55.]
 [77. 69. 62. 50.]
 [77. 69. 62. 50.]
 [77. 69. 62. 50.]
 [77. 69. 62. 50.]
 [77. 70. 62. 55.]
 [77. 70. 62. 55.]
 [77. 69. 62. 55.]
 [77. 69. 62. 55.]
 [75. 67. 63. 48.]
 [75. 67. 63. 48.]
 [75. 69. 63. 48.]
 [75. 69. 63. 48.]
 [74. 70. 65. 46.]
 [74. 70. 65. 46.]
 [74. 70. 65. 46.]
 [74. 70. 65. 46.]
 [72. 69. 65. 53.]
 [72. 69. 65. 53.]
 [72. 69. 65. 53.]
 [72. 69. 65. 53.]
 [72. 69. 65. 53.]
 [72. 69. 65. 53.]]
[[77.85289 77.10733 73.91023 73.91752]]
expected:  [72. 69. 65. 53.]


In [72]:
# choose a number of time steps
n_steps = None
# convert into input/output
i=1
n=1
x_input = np.array([dataXScaler[0]])

x_input = x_input.reshape((1, len(x_input), n_features))
while i<len(dataXScaler):
  # demonstrate prediction
  print('Input: ')
  for input in x_input:
    print(scaler.inverse_transform(input))
  print('---------------')
  yhat = model_gru.predict(x_input, verbose=1)
  print('Predicted Output: ', scaler.inverse_transform(yhat))
  print('expected: ', scaler.inverse_transform(dataYScaler)[i])
  print('\n\n')

  yhat = yhat.reshape((1, len(yhat), n_features))
  x_input = np.concatenate([x_input, yhat], axis=1)
  i += 1
  if i>20:
    break

Input: 
[[74. 70. 65. 58.]]
---------------
1/1 [==============================] - 0s 5ms/step
Predicted Output:  [[74.44904 69.60308 62.69999 57.07506]]
expected:  [74. 70. 65. 58.]



Input: 
[[74.         70.         65.         58.        ]
 [74.44903803 69.60308254 62.69998932 57.07506287]]
---------------
1/1 [==============================] - 0s 4ms/step
Predicted Output:  [[76.25694  72.169205 64.44614  62.029022]]
expected:  [74. 70. 65. 58.]



Input: 
[[74.         70.         65.         58.        ]
 [74.44903803 69.60308254 62.69998932 57.07506287]
 [76.25694132 72.16920662 64.44613743 62.02902424]]
---------------
1/1 [==============================] - 0s 4ms/step
Predicted Output:  [[76.527115 73.45797  64.80749  62.385166]]
expected:  [75. 70. 58. 55.]



Input: 
[[74.         70.         65.         58.        ]
 [74.44903803 69.60308254 62.69998932 57.07506287]
 [76.25694132 72.16920662 64.44613743 62.02902424]
 [76.5271138  73.45796943 64.8074832  62.38516545]]
----